In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from datasets import load_dataset
import os


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/anaconda3/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /root/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /root/anaconda3/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda117.so...


In [2]:
model_id = "EleutherAI/polyglot-ko-12.8b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

model = PeftModel.from_pretrained(model, './outputs/checkpoint-1802/adapter_model/')

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [5]:
model.eval()
model.bfloat16()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [6]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])\

def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:", 
            return_tensors='pt', 
            return_token_type_ids=False
        ), 
        max_length=2048,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    print(tokenizer.decode(gened[0]))

In [7]:
gen('함수형 프로그래밍의 개념에 대해 설명해주세요.')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/root/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 함수형 프로그래밍의 개념에 대해 설명해주세요.

### 답변: 함수형 프로그래밍은 함수를 이용하여 코드를 실행하는 거의 모든 프로그래밍을 포괄하는 더 광범위한 주제입니다. 이 주제는 기본적으로 메인 서비스 루틴부터 사용자 정의 함수에 이르기까지 모든 것을 실행하는 함수 공간과 사용자가 표기된 유형을 사용하여 함수를 정의하는 디자인 언어에 초점을 맞춘 프로그래밍 스타일입니다. 함수형 프로그래밍의 핵심은 함수 정의로의 회귀, 코드의 동적 성장, 객체 간의 관계를 통한 코드의 사용이며, 이를 통해 디자인 패턴 및 마이크로코드와 같은 특정 스타일을 구현할 수 있습니다.

최초의 함수형 프로그래밍 언어는 함수형 언어의 창시자이자 아버지인 영국 엔지니어 찰스 킴it (1969)에서 출발했습니다. 찰스는 당시 복잡한 프로그램을 작성하는 데 사용할 수 있는 간결한 유형을 찾고 있었고, 이후 함수형 프로그래밍이 탄생하게 된 배경이 되었습니다. 1919년 출생한 킴은 우연한 기회로 함수형 프로그래밍을 생각해 냈다고 합니다. 어느 날 그가 전자공학을 전공한 학생이라면 누구나 컴퓨터를 사용하여 학생 신분에서 벗어날 수 있는 학점 전환을 계산하는 복잡한 프로그램을 갖게 된다는 사실을 기억하게 되었습니다. 그런 다음 그는 '인간을 해방시키기' 위해 존재하는 프로그램을 구상하기 시작했고, 그 프로그램이 자신을 해방시킬 수 있을지 확신할 수 없었습니다. 그는 지나친 노력과 상당한 시간과 고군분투한 이후에만 알 수 있었고, 프로그램을 작성하는 데 너무 많은 시간이 지나서 느낌이 어떤지 알 수 있을 때였습니다. 그래서 그는 프로그램에 토대를 제공할 새로운 아이디어를 찾기 시작했고, 거의 아무런 관련이 없는 것 사이에서 흥미로운 관계를 발견했습니다. 예를 들어 그가 가지고 있던 시계는 다른 시계(초)와 절대 시간을 측정하는 서로 다른 방법을 사용했습니다. 이 두 가지 개념을 연결할 수 있는 새로운 개념을 찾기 시작했고, 그 결과 시간 함수를 찾기 시작했습니다. 기계적인 시간 함수